# MCPify your AWS Lambda Functions
## Transform multiple AWS Lambda functions into secure MCP tools with Bedrock AgentCore Gateway

## Overview
Bedrock AgentCore Gateway provides customers a way to turn their existing AWS Lambda functions into fully-managed MCP servers without needing to manage infra or hosting. Gateway will provide a uniform Model Context Protocol (MCP) interface across all these tools. Gateway employs a dual authentication model to ensure secure access control for both incoming requests and outbound connections to target resources. The framework consists of two key components: Inbound Auth, which validates and authorizes users attempting to access gateway targets, and Outbound Auth, which enables the gateway to securely connect to backend resources on behalf of authenticated users. Gateways uses IAM role to authorize the calls to AWS Lambda functions for outbound authorization.

![How does it work](images/lambda-iam-gateway.png)

### Tutorial Details


| Information          | Details                                                   |
|:---------------------|:----------------------------------------------------------|
| Tutorial type        | Interactive                                               |
| AgentCore components | AgentCore Gateway, AgentCore Identity                     |
| Agentic Framework    | Strands Agents                                            |
| Gateway Target type  | Multiple AWS Lambda functions                             |
| Inbound Auth IdP     | Amazon Cognito                                            |
| Outbound Auth        | AWS IAM                                                   |
| LLM model            | Anthropic Claude Sonnet 3.7, Amazon Nova Pro              |
| Tutorial components  | Creating AgentCore Gateway and Invoking AgentCore Gateway |
| Tutorial vertical    | Telecom/5G Network Optimization                          |
| Example complexity   | Intermediate                                              |
| SDK used             | boto3                                                     |

In the first part of the tutorial we will create multiple AgentCore Gateway targets

### Tutorial Architecture
In this tutorial we will transform multiple telecom optimization operations defined in AWS lambda functions into MCP tools and host them in Bedrock AgentCore Gateway.
For demonstration purposes, we will use a Strands Agent using Amazon Bedrock models
In our example we will use multiple telecom optimization tools from your lambda_zips directory.

## Prerequisites

To execute this tutorial you will need:
* Jupyter notebook (Python kernel)
* uv
* AWS credentials
* Amazon Cognito

## Configuring Authentication for Incoming AgentCore Gateway Requests
AgentCore Gateway provides secure connections via inbound and outbound authentication. For the inbound authentication, the AgentCore Gateway analyzes the OAuth token passed during invocation to decide allow or deny the access to a tool in the gateway. If a tool needs access to external resources, the AgentCore Gateway can use outbound authentication via API Key, IAM or OAuth Token to allow or deny the access to the external resource.



During the inbound authorization flow, an agent or the MCP client calls an MCP tool in the AgentCore Gateway adding an OAuth access token (generated from the user's IdP). AgentCore Gateway then validates the OAuth access token and performs inbound authorization.

If the tool running in AgentCore Gateway needs to access external resources, OAuth will retrieve credentials of downstream resources using the resource credential provider for the Gateway target. AgentCore Gateway pass the authorization credentials to the caller to get access to the downstream API. 

In [ ]:
# !pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# Set AWS credentials if not using Amazon SageMaker notebook
import os
# os.environ['AWS_ACCESS_KEY_ID'] = '' # Set the access key
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # Set the secret key
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1') # set the AWS region

In [ ]:
import os
import sys

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# Now you can import utils
import utils

In [ ]:
import boto3
import botocore
import glob
import os
import time
import json

# Create Lambda Execution Role
def create_lambda_execution_role():
    iam = boto3.client('iam')
    role_name = 'gateway_lambda_iamrole'
    
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Principal": {"Service": "lambda.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }]
    }
    
    try:
        response = iam.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy)
        )
        iam.attach_role_policy(
            RoleName=role_name,
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )
        return response['Role']['Arn']
    except iam.exceptions.EntityAlreadyExistsException:
        response = iam.get_role(RoleName=role_name)
        return response['Role']['Arn']

# Create the Lambda execution role
lambda_role_arn = create_lambda_execution_role()
print(f"Lambda role ARN: {lambda_role_arn}")

# Wait for role to propagate
print("Waiting for role to propagate...")
time.sleep(10)

def create_unique_lambda(zip_path, function_name):
    """Create Lambda with unique name"""
    lambda_client = boto3.client('lambda')
    
    try:
        with open(zip_path, 'rb') as f:
            lambda_code = f.read()
        
        response = lambda_client.create_function(
            FunctionName=function_name,
            Role=lambda_role_arn,
            Runtime='python3.12',
            Handler='lambda_function.lambda_handler',
            Code={'ZipFile': lambda_code}
        )
        return {'lambda_function_arn': response['FunctionArn'], 'exit_code': 0}
    except botocore.exceptions.ClientError as error:
        print(f":x: Error for {function_name}: {error}")
        if error.response['Error']['Code'] == 'ResourceConflictException':
            # Delete and recreate to get updated code
            try:
                lambda_client.delete_function(FunctionName=function_name)
                print(f"Deleted existing {function_name}")
                time.sleep(5)
                
                # Recreate with new code
                response = lambda_client.create_function(
                    FunctionName=function_name,
                    Role=lambda_role_arn,
                    Runtime='python3.12',
                    Handler='lambda_function.lambda_handler',
                    Code={'ZipFile': lambda_code}
                )
                return {'lambda_function_arn': response['FunctionArn'], 'exit_code': 0}
            except Exception as delete_error:
                print(f"Error recreating {function_name}: {delete_error}")
                return {'lambda_function_arn': 'Failed', 'exit_code': 1}
        else:
            return {'lambda_function_arn': 'Failed', 'exit_code': 1}
    except Exception as e:
        print(f":x: Unexpected error for {function_name}: {e}")
        return {'lambda_function_arn': 'Failed', 'exit_code': 1}

# Deploy functions
priority_functions = [
    "configure_slice_parameters",
    "create_v2x_handover_plan",
    "create_energy_saving_plan",
    "create_mimo_optimization_plan",
    "create_uav_resource_plan",
    "execute_energy_optimization",
    "execute_mimo_optimization",
    "execute_v2x_handover_optimization",
    "deploy_a1_policy",
    "execute_uav_resource_allocation"
]

zip_files = glob.glob("lambda_zips/*.zip")
selected_zips = [z for z in zip_files if os.path.splitext(os.path.basename(z))[0] in priority_functions]
deployed_functions = {}

print(f"Deploying {len(selected_zips)} priority functions...")

for zip_path in selected_zips:
    function_name = os.path.splitext(os.path.basename(zip_path))[0]
    lambda_resp = create_unique_lambda(zip_path, function_name)
    
    if lambda_resp and lambda_resp['exit_code'] == 0:
        deployed_functions[function_name] = lambda_resp['lambda_function_arn']
        print(f":white_check_mark: {function_name}")

print(f"Deployed {len(deployed_functions)} Lambda functions")

In [ ]:
#### Create an IAM role for the Gateway to assume
import utils
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

In [ ]:
def update_agentcore_gateway_role_trust_policy(new_trust_policy):
    iam_client = boto3.client('iam')
    role_name = "agentcore-sample-lambdagateway-role"
    
    iam_client.update_assume_role_policy(
        RoleName=role_name,
        PolicyDocument=json.dumps(new_trust_policy)
    )

new_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AssumeRolePolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock-agentcore.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Update the trust policy
update_agentcore_gateway_role_trust_policy(new_trust_policy)

# Create Amazon Cognito Pool for Inbound authorization to Gateway

In [ ]:
# Creating Cognito User Pool 
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Get discovery URL  
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# Create the Gateway with Amazon Cognito Authorizer for inbound authorization

In [ ]:
# CreateGateway with Cognito authorizer without CMK. Use the Cognito user pool created in the previous step
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # Client MUST match with the ClientId configured in Cognito
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='SampleLambdaGateway1',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway 
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='Sample AgentCore Gateway with Lambda Target'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# Create multiple AWS Lambda targets and transform into MCP tools

In [ ]:
# Tool schemas with correct parameters for each function
TOOL_SCHEMAS = {
    "create_v2x_handover_plan": {
        "name": "create_v2x_plan",
        "description": "Create dynamic handover plan for V2X based on vehicle context",
        "inputSchema": {
            "type": "object",
            "properties": {
                "vehicle_trajectory": {"type": "object", "description": "Vehicle trajectory information"},
                "mobility_context": {"type": "object", "description": "Mobility context data"}
            },
            "required": ["vehicle_trajectory", "mobility_context"]
        }
    },
    "execute_v2x_handover_optimization": {
        "name": "exec_v2x_opt", 
        "description": "Execute V2X handover optimization using AI/ML",
        "inputSchema": {
            "type": "object",
            "properties": {
                "plan_id": {"type": "string", "description": "Plan ID to execute"}
            },
            "required": ["plan_id"]
        }
    },
    "create_uav_resource_plan": {
        "name": "create_uav_plan",
        "description": "Create flight path based UAV resource allocation plan", 
        "inputSchema": {
            "type": "object",
            "properties": {
                "flight_path": {"type": "array", "description": "UAV flight path coordinates"},
                "uav_requirements": {"type": "object", "description": "UAV resource requirements"}
            },
            "required": ["flight_path", "uav_requirements"]
        }
    },
    "execute_uav_resource_allocation": {
        "name": "exec_uav_alloc",
        "description": "Execute UAV resource allocation with predictive beamforming",
        "inputSchema": {
            "type": "object", 
            "properties": {
                "plan_id": {"type": "string", "description": "Plan ID to execute"}
            },
            "required": ["plan_id"]
        }
    },
    "deploy_a1_policy": {
        "name": "deploy_a1",
        "description": "Deploy A1 policy for network management",
        "inputSchema": {
            "type": "object",
            "properties": {
                "policy_type": {"type": "string"},
                "s_nssai": {"type": "string"},
                "policy_params": {"type": "object"}
            },
            "required": ["policy_type", "s_nssai", "policy_params"]
        }
    },
    "configure_slice_parameters": {
        "name": "config_slice",
        "description": "Configure network slice parameters",
        "inputSchema": {
            "type": "object",
            "properties": {
                "node_id": {"type": "string"},
                "s_nssai": {"type": "string"},
                "slice_config": {"type": "object"}
            },
            "required": ["node_id", "s_nssai", "slice_config"]
        }
    },
    "create_energy_saving_plan": {
        "name": "create_energy_plan",
        "description": "Create energy saving optimization plan",
        "inputSchema": {
            "type": "object", 
            "properties": {
                "network_topology": {"type": "object"},
                "traffic_patterns": {"type": "object"}
            },
            "required": ["network_topology", "traffic_patterns"]
        }
    },
    "execute_energy_optimization": {
        "name": "exec_energy_opt",
        "description": "Execute energy optimization plan",
        "inputSchema": {
            "type": "object",
            "properties": {
                "plan_id": {"type": "string", "description": "Plan ID to execute"}
            },
            "required": ["plan_id"]
        }
    },
    "create_mimo_optimization_plan": {
        "name": "create_mimo_plan",
        "description": "Create MIMO optimization plan",
        "inputSchema": {
            "type": "object",
            "properties": {
                "antenna_config": {"type": "object"},
                "channel_conditions": {"type": "object"}
            },
            "required": ["antenna_config", "channel_conditions"]
        }
    },
    "execute_mimo_optimization": {
        "name": "exec_mimo_opt", 
        "description": "Execute MIMO optimization",
        "inputSchema": {
            "type": "object",
            "properties": {
                "plan_id": {"type": "string", "description": "Plan ID to execute"}
            },
            "required": ["plan_id"]
        }
    }
}

# Create gateway targets with proper schemas
created_targets = []

for func_name, lambda_arn in deployed_functions.items():
    tool_schema = TOOL_SCHEMAS.get(func_name, {
        "name": func_name,
        "description": f"Tool for {func_name.replace('_', ' ')}",
        "inputSchema": {"type": "object", "properties": {"test": {"type": "string"}}}
    })
    
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": lambda_arn,
                "toolSchema": {"inlinePayload": [tool_schema]}
            }
        }
    }

    credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]
    targetname = func_name.replace('_', '-') + '-target'
    
    response = gateway_client.create_gateway_target(
        gatewayIdentifier=gatewayID,
        name=targetname,
        description=f'Lambda Target for {func_name}',
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=credential_config
    )
    
    created_targets.append(targetname)
    print(f"Created target: {targetname}")
    time.sleep(10)  # Increased delay to avoid rate limiting

print(f"Created {len(created_targets)} gateway targets")

# Calling Bedrock AgentCore Gateway from a Strands Agent

The Strands agent seamlessly integrates with AWS tools through the Bedrock AgentCore Gateway, which implements the Model Context Protocol (MCP) specification. This integration enables secure, standardized communication between AI agents and AWS services.

At its core, the Bedrock AgentCore Gateway serves as a protocol-compliant Gateway that exposes fundamental MCP APIs: ListTools and InvokeTools. These APIs allow any MCP-compliant client or SDK to discover and interact with available tools in a secure, standardized way. When the Strands agent needs to access AWS services, it communicates with the Gateway using these MCP-standardized endpoints.

The Gateway's implementation adheres strictly to the (MCP Authorization specification)[https://modelcontextprotocol.org/specification/draft/basic/authorization], ensuring robust security and access control. This means that every tool invocation by the Strands agent goes through authorization step, maintaining security while enabling powerful functionality.

For example, when the Strands agent needs to access MCP tools, it first calls ListTools to discover available tools, then uses InvokeTools to execute specific actions. The Gateway handles all the necessary security validations, protocol translations, and service interactions, making the entire process seamless and secure.

This architectural approach means that any client or SDK that implements the MCP specification can interact with AWS services through the Gateway, making it a versatile and future-proof solution for AI agent integrations.

![Strands agent calling Gateway](images/strands-lambda-gateway.png)

# Request the access token from Amazon Cognito for inbound authorization

In [ ]:
import time
time.sleep(10)

In [ ]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

# Strands agent calling MCP tools of multiple AWS Lambda functions using Bedrock AgentCore Gateway

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## The IAM credentials configured in ~/.aws/credentials should have access to Bedrock model
yourmodel = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.7,
)

In [ ]:

# Test the updated Lambda function directly
import boto3
import json

lambda_client = boto3.client('lambda')

test_payload = {
    "vehicle_trajectory": {"speed": 60, "direction": "north"},
    "mobility_context": {"cell_id": "cell_001", "signal_strength": -80}
}

response = lambda_client.invoke(
    FunctionName='create_v2x_handover_plan',
    Payload=json.dumps(test_payload)
)

result = json.loads(response['Payload'].read())
print(f"Direct Lambda test result: {result}")

In [ ]:
from strands import Agent
import logging

# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools 
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(model=yourmodel, tools=tools)
    print(f"Tools loaded in the agent are {agent.tool_names}")
    
    # Test with proper telecom parameters
    agent("Hi, can you list all tools available to you")
    
    # Test V2X handover plan creation with correct parameters
    response = agent("""Create a V2X handover plan with these parameters:
    - vehicle_trajectory: {"speed": 60, "direction": "north", "location": {"lat": 40.7128, "lng": -74.0060}}
    - mobility_context: {"cell_id": "cell_001", "signal_strength": -80, "handover_threshold": -90}
    """)

    print(response)
    
    # # Direct tool call with correct parameters
    # if agent.tool_names:
    #     # Find V2X tool
    #     v2x_tool = next((t for t in agent.tool_names if "v2x-handover" in t), agent.tool_names[0])
        
    #     result = client.call_tool_sync(
    #         tool_use_id="test-v2x-001",
    #         name=v2x_tool,
    #         arguments={
    #             "vehicle_trajectory": {"speed": 60, "direction": "north"},
    #             "mobility_context": {"cell_id": "cell_001", "signal_strength": -80}
    #         }
    #     )
    #     print(f"Tool Call result: {result['content'][0]['text']}")

    # agent = Agent(model=yourmodel, tools=tools)


In [ ]:
with client:
    # Create comprehensive telecom optimization plan
    workflow_response = agent("""
    I need to optimize a telecom network for a smart city deployment. Please help me:
    
    1. Create a V2X handover plan for autonomous vehicles with:
       - Vehicle trajectory: speed 80 km/h, direction northeast, urban environment
       - Mobility context: high-density cell coverage, signal strength -75 dBm
    
    2. Set up UAV resource allocation for emergency services with:
       - Flight path covering downtown area at 150m altitude
       - Requirements: 50Mbps bandwidth, 2ms latency for critical communications
    
    3. Deploy an A1 policy for QoS management:
       - Policy type: Emergency_QoS
       - Network slice: emergency services (01-000002)
       - Parameters: highest priority, guaranteed 100Mbps bandwidth
    """)
    
    print("Workflow Response:", workflow_response)

**Issue: if you get below error while executing below cell, it indicates incompatibily between pydantic and pydantic-core versions.**

```
TypeError: model_schema() got an unexpected keyword argument 'generic_origin'
```
**How to resolve?**

You will need to make sure you have pydantic==2.7.2 and pydantic-core 2.27.2 that are both compatible. Restart the kernel once done.